Домашнее задание 2 будет более свободным по форме:

собрать небольшой корпус текстов в формате *.txt
обучить и протестировать векторную модель любым способом Word2Vec (CBOW, Skip-gram), GloVe, FastText
можно использовать gensim, можно обучать W2V с нуля (если вы достаточно опытные) и т.д.
Главное, чтобы "на выходе" у вас была своя обученная модель, несколько примеров семантически близких слов и 1 визуалка

In [1]:
#устанавливаем и обновляем
!pip install gensim
!pip install numpy
!pip install --upgrade numpy
!pip install --no-cache-dir --force-reinstall gensim

  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
  Using cached numpy-2.2.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you

In [1]:
#импорт и загрузка
import nltk
import re
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from nltk.corpus import stopwords
from nltk import sent_tokenize, word_tokenize
from pymorphy3 import MorphAnalyzer
from sklearn.manifold import TSNE
from pprint import pprint as print
from gensim.models.fasttext import FastText

nltk.download('stopwords')
nltk.download('punkt_tab')
stop_words = set(stopwords.words('russian'))
morph = MorphAnalyzer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:
#взяла тексты из старой папки бакалавриата :)
file_names = [
    'братья_карамазовы_1.txt',
    'братья_карамазовы_2.txt',
    'братья_карамазовы_4.txt',
    'братья_карамазовы_4.txt'
]

texts = ''
for file_name in file_names:
    with open(file_name, 'r', encoding='utf-8') as file:
        texts += file.read()
print(texts[:150])

('\n'
 '   Начиная жизнеописание героя моего, Алексея Федоровича Карамазова, '
 'нахожусь в некотором недоумении. А именно: хотя я и называю Алексея '
 'Федоровича м')


In [3]:
def normalize_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r'[^\w\s]', '', sentence)
    return sentence

def lemmatize_word(word):
    try:
        return morph.parse(word)[0].normal_form if word else word
    except:
        return word

def preprocess_text(texts):
    tokenized_corpus = []
    for sentence in sent_tokenize(texts):
        normalized_sentence = normalize_sentence(sentence)
        tokens = [
            lemmatize_word(token)
            for token in word_tokenize(normalized_sentence)
            if token and token not in stop_words
        ]
        if tokens:  # добавляем только непустые списки токенов
            tokenized_corpus.append(tokens)

    return tokenized_corpus

corp = preprocess_text(texts)
print(corp[:100])

[['начинать',
  'жизнеописание',
  'герой',
  'мой',
  'алексей',
  'фёдорович',
  'карамазов',
  'находиться',
  'некоторый',
  'недоумение'],
 ['именно',
  'хотя',
  'называть',
  'алексей',
  'фёдорович',
  'мой',
  'герой',
  'однако',
  'знать',
  'человек',
  'отнюдь',
  'великий',
  'посему',
  'предвидеть',
  'неизбежный',
  'вопрос',
  'род',
  'таковой',
  'замечательный',
  'ваш',
  'алексей',
  'фёдорович',
  'выбрать',
  'свой',
  'герой'],
 ['сделать', 'такой'],
 ['кто', 'известный'],
 ['почему',
  'читатель',
  'должный',
  'тратить',
  'время',
  'изучение',
  'факт',
  'жизнь'],
 ['последний',
  'вопрос',
  'самый',
  'роковой',
  'ибо',
  'мочь',
  'лишь',
  'ответить',
  'увидеть',
  'сам',
  'роман'],
 ['коль',
  'прочесть',
  'роман',
  'увидеть',
  'согласиться',
  'примечательность',
  'мой',
  'алексей',
  'фёдорович'],
 ['говорить', 'прискорбие', 'это', 'предвидеть'],
 ['примечательный',
  'решительно',
  'сомневаться',
  'успеть',
  'это',
  'доказать',
  'чит

In [35]:
from gensim.models import FastText
from gensim.models.callbacks import CallbackAny2Vec
from tqdm import tqdm

# для отображения прогресс-бара эпох
class EpochProgress(CallbackAny2Vec):
    def __init__(self, total_epochs):
        self.pbar = tqdm(total=total_epochs, desc="Epochs", unit="epoch")
        self.epoch = 0

    def on_epoch_end(self, model):
        self.epoch += 1
        self.pbar.update(1)

    def on_train_end(self, model):
        self.pbar.close()

# папраметры
model_params = {
    'vector_size': 100,
    'window': 10,
    'min_count': 3,
    'sg': 1,
    'negative': 15,
    'ns_exponent': 0.75,
    'min_n': 3,
    'max_n': 6,
    'epochs': 50
}

# инициализируем
model_tokenized = FastText(**{k: v for k, v in model_params.items() if k != 'epochs'})

#делаем словарь
model_tokenized.build_vocab(corpus_iterable=corp, progress_per=10000)

# прогресс бар инициализируем
epoch_progress = EpochProgress(total_epochs=model_params['epochs'])

# обучение
model_tokenized.train(
    corpus_iterable=corp,
    epochs=model_params['epochs'],
    total_examples=model_tokenized.corpus_count,
    total_words=model_tokenized.corpus_total_words,
    callbacks=[epoch_progress],
    compute_loss=True,
    start_alpha=0.025,
    end_alpha=0.001
)

print(f"Done")

Epochs: 100%|██████████| 50/50 [04:05<00:00,  4.92s/epoch]

'Done'


In [38]:
similar_words = model_tokenized.wv.most_similar('война', topn=5)

# Выводим результат
print("Семантически близкие слова:")
for word, similarity in similar_words:
    print(f"{word}: {similarity}")

'Семантически близкие слова:'
'дипломат: 0.695756733417511'
'зарождаться: 0.689919114112854'
'искусство: 0.6736543774604797'
'разбойник: 0.6547515392303467'
'представление: 0.6449184417724609'


Вывод: в целом какой-то смысл в этих схожестях есть, но конечно хотелось бы лучше. я пыталась через увеличение эпох, размера вектора, через грид параметров, но в целом как-то уровень точности оставался прежним. Вероятно из-за большого объема текста не очень эффективно получается?